In [ ]:
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers "trl<0.9.0" peft accelerate bitsandbytes
!pip install torch==2.4.0

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-8o02qa3p/unsloth_4f3ad2b2c3e743d998775330ff089bd4
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-8o02qa3p/unsloth_4f3ad2b2c3e743d998775330ff089bd4
  Resolved https://github.com/unslothai/unsloth.git to commit 52bc19d1fa4cd3557b785127fd68b5f4d1c34347
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Us

In [ ]:
# # Load model directly
# from transformers import AutoTokenizer, AutoModelForCausalLM

# tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2-1.5B")
# model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen2-1.5B")

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048
dtype = None
load_in_4bit = True
fourbit_models = [
    "unsloth/Qwen2-1.5b-bnb-4bit",
]

# https://huggingface.co/unsloth/Qwen2-1.5B
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen2-1.5B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth 2024.8: Fast Qwen2 patching. Transformers = 4.44.0.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.27.post2. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/1.14G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/167 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.51k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/107 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/256 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

Unsloth 2024.8 patched 28 layers with 0 QKV layers, 28 O layers and 28 MLP layers.


In [ ]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
### Instruction:
{}
### Input:

### Response:
{}"""
EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["prompt"]
    # inputs       = examples.get("input", "")
    outputs      = examples["response"]
    texts = []
    for instruction, output in zip(instructions, outputs):
        text = alpaca_prompt.format(instruction, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }

In [ ]:
from datasets import load_dataset
dataset = load_dataset("nvidia/HelpSteer", split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True,)

Map:   0%|          | 0/35331 [00:00<?, ? examples/s]

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 8,
        # Use num_train_epochs = 1, warmup_ratio for full training runs!
        warmup_steps = 20,
        max_steps = 120,
        learning_rate = 5e-5,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

Map (num_proc=2):   0%|          | 0/35331 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 35,331 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 8
\        /    Total batch size = 16 | Total steps = 120
 "-____-"     Number of trainable parameters = 18,464,768


Step,Training Loss
1,2.281800
2,2.012400
3,2.183800
4,2.179300
5,2.240400
6,2.061200
7,2.174500
8,2.225600
9,2.009500
10,2.152900


In [ ]:
!zip -r /content/qwen2-1.5B-finetune.zip /content/outputs/checkpoint-120

  adding: content/outputs/checkpoint-120/ (stored 0%)
  adding: content/outputs/checkpoint-120/adapter_config.json (deflated 54%)
  adding: content/outputs/checkpoint-120/optimizer.pt (deflated 16%)
  adding: content/outputs/checkpoint-120/adapter_model.safetensors (deflated 8%)
  adding: content/outputs/checkpoint-120/merges.txt (deflated 57%)
  adding: content/outputs/checkpoint-120/tokenizer.json (deflated 72%)
  adding: content/outputs/checkpoint-120/README.md (deflated 66%)
  adding: content/outputs/checkpoint-120/vocab.json (deflated 61%)
  adding: content/outputs/checkpoint-120/added_tokens.json (deflated 39%)
  adding: content/outputs/checkpoint-120/special_tokens_map.json (deflated 58%)
  adding: content/outputs/checkpoint-120/scheduler.pt (deflated 57%)
  adding: content/outputs/checkpoint-120/training_args.bin (deflated 51%)
  adding: content/outputs/checkpoint-120/trainer_state.json (deflated 79%)
  adding: content/outputs/checkpoint-120/rng_state.pth (deflated 25%)
  addin

## Evaluate

In [ ]:
from datasets import load_dataset
dataset = load_dataset("nvidia/HelpSteer", split = "validation")
# dataset = dataset.map(formatting_prompts_func, batched = True,)

In [ ]:
trainer.evaluate()

1789

In [ ]:
trainer.save_model("content/qwen2-1.5B")

In [20]:
from huggingface_hub import notebook_login

notebook_login()

In [22]:
trainer.push_to_hub("huynd/qwen2-1.5B-HelpSteer")

CommitInfo(commit_url='https://huggingface.co/huynd/outputs/commit/2fce8e9fe9b297f4420648af77090908e763d6f8', commit_message='huynd/qwen2-1.5B-HelpSteer', commit_description='', oid='2fce8e9fe9b297f4420648af77090908e763d6f8', pr_url=None, pr_revision=None, pr_num=None)